### 스마트 냉장고
* 데이터 셋 : 과채류 30 종류
* 피쳐/속성 : 그림 파일
* 타겟/라벨 : 폴더명
* 학습-방법 : 지도학습 > 분류 > 다중분류
* 알고리즘 : 
* 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
<hr>

In [1]:
# ---------------------------------------------------------------------
# 모델링 관련 모듈 로딩
# ---------------------------------------------------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchinfo import summary
from torch.utils.data import random_split

from torchmetrics.classification import accuracy
from torchmetrics.classification import MulticlassF1Score, MulticlassAccuracy, MulticlassConfusionMatrix
import torch.optim.lr_scheduler as lr_scheduler

import torchvision.models as models

# ---------------------------------------------------------------------
# 데이터 분석 관련 모듈 로딩
# ---------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------------------------
# 이미지 관련 모듈 로딩
# ---------------------------------------------------------------------
import cv2
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2

# ---------------------------------------------------------------------
# 기타 모듈 로딩
# ---------------------------------------------------------------------
import time
import os

# 활용 패키지 버전 체크
print(f'torch Ver.:{torch.__version__}')
print(f'pandas Ver.:{pd.__version__}')
print(f'numpy Ver.:{np.__version__}')

torch Ver.:2.4.1
pandas Ver.:2.0.3
numpy Ver.:1.24.3


In [2]:
# DEVICE 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

* 이미지 데이터셋 생성
<hr>

In [3]:
transConvert = transforms.Compose([transforms.Resize([236]), transforms.CenterCrop(224),transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [4]:
base_dir = r'C:\Users\KDP-14\Desktop\VSCode\LocalData\self_project_image\fruit_veget_image'  # 원본 데이터가 있는 경로
train_dir = r'C:\Users\KDP-14\Desktop\VSCode\LocalData\self_project_image\train'   # 훈련 데이터를 저장할 경로
test_dir = r'C:\Users\KDP-14\Desktop\VSCode\LocalData\self_project_image\test'     # 테스트 데이터를 저장할 경로

In [7]:
from sklearn.model_selection import train_test_split
import shutil

class_names = os.listdir(base_dir)

for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    
    # 클래스 내의 모든 이미지 파일 목록 가져오기
    images = [img for img in os.listdir(class_dir) if img.endswith(('.jpg', '.png'))]

    # 훈련과 테스트 세트로 나누기 (80% 훈련, 20% 테스트)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # 훈련 및 테스트 데이터 이동
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    for image in train_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(train_dir, class_name, image))

    for image in test_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(test_dir, class_name, image))
